# [Sensifai](https://sensifai.com) Image Tagging
Sensifai offers one of the most accurate Deep Learning training platform to train your image recognition system and incorporate it into your application. This product lets you access Sensifai's advanced image recognition algorithm and train it with our own data and validate it with any set you would like. 

## Step 1: preparing your data
Before starting the training job you have to prepare your data and copy the files to S3. Our training algorithm only supports image data formats. you can simply prepare your data by following one of the methods below:

- if you have a validation set, copy images of train/validation in separate train/validation folders. also, you need to prepare separate label files in CSV format for your data splits(train.csv, validation.csv) and copy them under their corresponding folders.
- if you do not provide a separate validation set, you can simply copy all your images in train folder and provide a labels.csv file. The algorithm will separate your data based on the min_val_samples parameter.

Our algorithm only supports image file formats for training, but for inference, you could either choose  image or a video version get the results. it's recommended to use images that satisfy the following  conditions:
- the algorithm supports most of the common image formats (jpg, jpeg,...) and we do not set any limitation on the image or video types, however, if the algorithm cannot detect a format, it skips the file.  also if the file is corrupted, the algorithm skips it. at training time, if the number of image files that algorithm skip exceeds 50% of all images in the folder the training procedure exited with the failure code.
- we do not have strict conditions on the resolution. However, very low-resolution images (lower than 224 * 224) may have a bad effect on training accuracy. also, very high-resolution images take longer time for transferring and preprocessing. 
- our multilabel algorithm can cope with unbalanced datasets. but it's recommended to have a minimum of ~100 images for a tag, to reach outstanding results.

### Label File format:

- you should provide a comma-separated CSV file with two columns: image_name, tags. for Multi-Label training, you need to separate tags with space. an example of a valid CSV file is shown below:

| image_name      | tags           |
| --------------- | -------------- |
| image_0001.jpg  | person         |
| image_0002.jpg  | tree sky grass |

- Make sure to name the files correctly(you should either have train.csv,validation.csv or labels.csv).
- It's important to have a balanced dataset in order to reach outstanding results.

## uploading files to S3

In [28]:
import sagemaker as sage
import boto3
import time
from sagemaker import get_execution_role

role = get_execution_role()

bucket = "your bucket here"
prefix = "prefix on s3 that the test files are stored"
sess = sage.Session()
s3_train="s3://{}/{}/train/".format(bucket,prefix)
s3_validation="s3://{}/{}/validation/".format(bucket,prefix)

#we already have transfer the data to s3, if you want to copy the files uncomment below code  
#s3_train = sess.upload_data(train_data_dir, bucket, "{}/train".format(prefix))
#s3_validation = sess.upload_data(validation_data_dir, bucket, "{}/validation".format(prefix))

print("uploaded training data file to {}".format(s3_train))
print("uploaded validation data file to {}".format(s3_validation))

uploaded training data file to s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/train/
uploaded validation data file to s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/validation/



### important notes:
- currently, we support just "FileMode" for input mode. In order to insure that there is enough space for transferring and preprocessing files, please set *ValumeSizeinGB* parameter of the *ResourceConfig* section to 2*size_of_dateset-inGB*)
- Choosing a more powerful system does not always lead to better results. Also, choosing a network with more layers is not always the best option. Select the number of layers and the instance type based on the size of dataset and type of input data.

## Step 2: Create a model 
__Training Parameters__


| Name                   | Description                                                                                       | IsRequired | IsTunable | DefaultValue | Range                     | Type        |
|------------------------|---------------------------------------------------------------------------------------------------|------------|-----------|--------------|---------------------------|-------------|
| min_val_samples        | minimum number of validation samples for each tag (if no val label file exists!)                  | False      | False     | 20           | [1, 1000]                 | Integer     |
| learning_rate          | Initial learning rate                                                                             | False      | False     | 0.001        | [0.000001, 0.1]           | Continuous  |
| momentum               | Momentum                                                                                          | False      | False     | 0.9          | [0, 0.9]                  | Continuous  |
| model_depth            | Number of layers for model                                                                        | False      | False     | 101          | 18, 34, 50, 101, 152, 154 | Categorical |
| settings               | select Training Setting                                                                           | True       | False     | Multi-Label  | Multi-Label, Single-Label | Categorical |
| batch_size             | batch size(if set to 0, will automatically set batch size considering GPU memories)               | False      | False     | 0            | [0, 500]                  | Integer     |
| lr_decay               | Factor by which the learning rate will be reduced. new_lr = lr * factor                           | False      | False     | 0.1          | [0.01, 0.9]               | Continuous  |
| lr_patience            | Patience of LR scheduler                                                                          | False      | False     | 5            | [1, 100]                  | Integer     |
| max_patience           | Terminate training after validation loss become greater than train loss for this number of epochs | False      | False     | 10           | [1, 500]                  | Integer     |
| n_epochs               | Total number of training epochs                                                                   | False      | False     | 30           | [1, 1000]                 | Integer     |
| result_second_interval | return results for tags in this second Intervals                                                  | False      | False     | 3            | [1, 10]                   | Integer     |
| thresholdValue         | Critical Parameter for Selecting Class Labels                                                     | False      | False     | 0.5          | [0.01, 0.95]              | Continuous  |
| num_result_tags        | Number of tags(Top n tags) to show in each timestamp of Inference json file                       | False      | False     | 5            | [1, 10]                   | Integer     |
| score_result_threshold | Show results greater than this threshold for each timestamp in Inference json file                | False      | False     | 0.5          | [0.0001, 0.99]            | Continuous  |

__Run a SageMaker training job__

This code will start a training job, wait for it to be done, and report its status.

In [30]:
%%time
alg_arn="COPY ALGOTITHM ARN HERE "
job_name_prefix = 'train-sensifai-image-tagging'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

create_training_params = \
{   "TrainingJobName": job_name,
    "AlgorithmSpecification": {
        "AlgorithmName":alg_arn,
        "TrainingInputMode": "File",
        "MetricDefinitions": []
    },
    "RoleArn": role,
    "EnableNetworkIsolation":True,
    "OutputDataConfig": {
        "S3OutputPath": 's3://{}/{}/output/{}'.format(bucket,prefix, job_name_prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p2.8xlarge",
        "VolumeSizeInGB": 40
    },
    "TrainingJobName": job_name,
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 14400
    },
    "HyperParameters": {
        "min_val_samples": "20",
        "model_depth": "101",
        "result_second_interval": "3",
        "settings": "Multi-Label", 
        "learning_rate": "0.01",
        "momentum": "0.5",
        "lr_decay": "0.1", 
        "batch_size": "0",
        "lr_patience": "5", 
        "max_patience": "10",
        "n_epochs": "10", 
        "thresholdValue": "0.5",
        "num_result_tags": "5",
        "score_result_threshold": "0.5"
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "",
            "CompressionType": "None"
        },
    ]
}

sagemaker = boto3.client(service_name='sagemaker')
sagemaker.create_training_job(**create_training_params)
status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

try:
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    job_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = job_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
except:
    print('Training failed to start')
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job current status: InProgress
Training job ended with status: Completed
CPU times: user 349 ms, sys: 21.2 ms, total: 370 ms
Wall time: 46min 3s


## Step3 :Create a SageMaker model 
This will set up the model created during training within SageMaker to be used later for recognition.


In [31]:
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name="sensifai-tagging" + timestamp
job_info = sagemaker.describe_training_job(TrainingJobName=job_name)
model_data = job_info['ModelArtifacts']['S3ModelArtifacts']

model_package_arn =  "Paste the model ARN"
model_creation = {
    "ModelName": model_name,
    "PrimaryContainer": {
        "ModelPackageName": model_package_arn
    },
    "ExecutionRoleArn":role,
    "EnableNetworkIsolation": True,
}

model = sagemaker.create_model(**model_creation)
sagemaker.describe_model(ModelName = model_name)

{'ModelArn': 'arn:aws:sagemaker:us-east-2:320478615219:model/sensifai-tagging-2018-11-14-15-40-19', 'ResponseMetadata': {'RequestId': 'd01eab0b-2360-469c-8985-1f1d320d1aae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd01eab0b-2360-469c-8985-1f1d320d1aae', 'content-type': 'application/x-amz-json-1.1', 'content-length': '98', 'date': 'Wed, 14 Nov 2018 15:40:19 GMT'}, 'RetryAttempts': 0}}


## step 4: inference with the trained model  (Batch transform)
finally the model is ready to serve and you can feed the videos to the model and save the results in output folder


In [32]:
%%time

s3_batch_input="s3://{}/{}/test/".format(bucket,prefix)
#we already have transfer the data to s3, if you want to copy the files uncomment below code  
# s3_bath_input = sess.upload_data(batch_input_dir, bucket, "{}/test".format(prefix))
print("uploaded batch data files to {}".format(s3_batch_input))

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "sensifai-tagging-bt" + timestamp
batch_output = 's3://{}/{}/output/{}'.format(bucket,prefix, batch_job_name)

request = \
{
  "TransformJobName": batch_job_name,
  "MaxConcurrentTransforms": 0,
  "MaxPayloadInMB": 0,
  "ModelName": model_name,
  "TransformInput": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": s3_batch_input
      }
    },
    "ContentType": "video/mp4",
    "CompressionType": "None",
    "SplitType": "None"
  },
  "TransformOutput": {
    "S3OutputPath": batch_output,
    "Accept": "application/json",
    "AssembleWith": "Line"
  },
  "TransformResources": {
    "InstanceType": "ml.p2.xlarge",
    "InstanceCount": 1
  }
}

sagemaker.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while(True):
    job_info = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = job_info['TransformJobStatus']
    if status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = job_info['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    time.sleep(30)

uploaded batch data files to s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/test/
Created Transform job with name:  sensifai-tagging-bt-2018-11-14-15-40-22
Transform job ended with status: Completed
CPU times: user 163 ms, sys: 5.52 ms, total: 168 ms
Wall time: 6min 31s



### download the results

In [33]:
import os
import json
from pprint import pprint

output_path="./output"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
!aws s3 cp $batch_output $output_path --recursive

#do anything with json files

download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/basket_shot.mp4.out to output/basket_shot.mp4.out
download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/image_0039.jpg.out to output/image_0039.jpg.out
download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/image_0059.jpg.out to output/image_0059.jpg.out
download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/image_0100.jpg.out to output/image_0100.jpg.out
download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/plant.jpg.out to output/plant.jpg.out
download: s3://sensifai-sagemaker-artifacts/algorithm-validation/Tagging/output/sensifai-tagging-bt-2018-11-14-15-40-22/image_0079.jpg.out to output/image_0079.jpg.out


## step 5 : cleaning up

In [ ]:
# optionally uncomment and run the code to clean everything up
#sagemaker.delete_model(ModelName= model_name)